# Estonia | Person: survived or not ?  

![](https://www.dw.com/image/49657763_304.jpg)

Hey guys, We all know about the Estonia Disaster. The Estonia disaster occurred on Wednesday, 28 September 1994, between about 00:55 and 01:50 (UTC+2). Today we are gonna to discuss about this disaster not the history of that but we are going to discuss on how many persons were alive in this disaster and how many are not. As come this dataset, there are six columns in that :
* PassengerID
* Country
* FirstName
* LastName
* Sex
* Age
* Category
* Survived

By the name of the columns we can easily understand what is the content and type of column. Supoose there is a Country column in that different countries are there from which countries people belonged. Hope you guys have an idea of that what we gonna to do it in that notebook. Right ?

As we need to import some packages. And these packages are very common in use. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [ ]:
data = pd.read_csv('../input/passenger-list-for-the-estonia-ferry-disaster/estonia-passenger-list.csv')

Looks like our dataset first and then we will decide whether there is a need of preprocessing or not.

In [ ]:
data.head()

Guys, Info part of data tells clearly about the dataset because it tells about the index, name of column, how many nulls values it have, and the most important part of the columns are its data type. 

In [ ]:
data.info()

In [ ]:
data.describe()

We can see that there are 989 values or rows and 8 columns. 

In [ ]:
data.shape

In [ ]:
data.tail()

By this dataset, there are 16 countries affected by this disaster and the name of these countries are as following :

In [ ]:
data.Country.unique()

In [ ]:
data.Sex.unique()

In the category column of dataset there are two classes i.e P and C. It means
* P - Passenger
* C - Crew

In [ ]:
data.Category.unique()

So there are five categorical variables our eight. And the name of these :
* Country
* Firstname
* Lastname
* Age
* Sex
* Category


Come to making model of predicting a person is survived or not it is clear Firstname and Lastname has no effect on the predictive column.

So we drop these columns by using drop function. 

In [ ]:
data =  data.drop(['Firstname', 'Lastname'], axis = 1)

Now our dataset looks like :

In [ ]:
data.head()

So guys we have to do some preprocessing in it because there are still some categorical variables in this. 

So let's start.

In [ ]:
label = LabelEncoder()
data["Country"] = label.fit_transform(data["Country"])
data["Sex"] = label.fit_transform(data["Sex"])
data["Category"] = label.fit_transform(data["Category"])

So now we have gone through the preprocessing part now our dataset is looks like :

In [ ]:
data.head()

So here is the quick look at the dataset and this is just a hist graph but by heatmap we clearly understand which feature has more importances.

In [ ]:
data.hist(figsize = (10, 10))
plt.show()

In [ ]:
print("Heatmap")
sns.heatmap(data.corr(), center = 0,  annot = True, linewidth = .5)
plt.show()

We have to divide the dataset into its features part and the labelling part. I divide it into X which has all features and y which has labels of the features.

In [ ]:
X = data.iloc[:, 0:5].values
y = data.iloc[:, 5].values

By this plot, we can see that most of the people are dead in this disaster and very few people are alive.

In [ ]:
sns.countplot(data["Survived"])

In [ ]:
X

So these are the shape of X and y which explain how many rows we are going to insert in the dataset.

In [ ]:

print("X array has Shape", X.shape)
print("Y array has shape", y.shape)

For doing modelling part, we have to do import some essential libraries. In this we are going to use the Random Forest Classifier and find its accuracy and F1 score.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

**Splittion of Dataset into TRAIN and TEST**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Guys, we are not going to use the library and find its accuarcy because it is very common method we all are used it. In this first I tuned some parameters and then by the tuned classifier we are going to find the accuarcy of that model.

Here is the **List of Parameters**

In [ ]:
params = {
    'n_estimators' : [60, 70, 100, 110, 120],
    'criterion'    : ['gini', 'entropy'],
    'max_depth'    : [1, 2, 3, 4],
    'min_samples_split' : [0.1, 0.7, 0.9, 1, 2],
    'min_samples_leaf' : [0.1, 0.6, 0.9, 1, 2],
    'min_weight_fraction_leaf' : [0.0, 0.1, 0.2, 0.3, 0.4],
    'max_features' : ['auto', 'sqrt', 'log2'],
    'max_leaf_nodes' : [1, 2, 3, 4, 5],
    'min_impurity_decrease' : [0.1, 0.0, 0.2, 0.3],
    'bootstrap' : [True, False],
    'oob_score' : [True, False],
    'verbose' : [0, 1, 2, 3],
    'warm_start' : [True, False],
    'class_weight' : ['balanced', 'balanced_subsample'],
    'max_samples' : [1, 2, 3, 4, 5]
}

By the function **timer** we can see the starting time and end time of tuning parameters.

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
classifier = RandomForestClassifier()

**Use the RandomSearchCV for tuning the classifier**

In [ ]:
random_search=RandomizedSearchCV(classifier,
                                 param_distributions=params,
                                 n_iter=10,
                                 scoring='roc_auc',
                                 n_jobs=10,
                                 cv=7,verbose=3)

Here we go. Tuning is in progress.

In [ ]:
start_time = timer(None) 
random_search.fit(X_train,y_train)
timer(start_time) 

In [ ]:
random_search.best_estimator_

**Best Tuned Parameters** :

In [ ]:
random_search.best_params_

Just call the classifier and find the accuracy of that.

In [ ]:
classifier = RandomForestClassifier(class_weight='balanced_subsample', max_depth=2,
                       max_leaf_nodes=4, max_samples=2,
                       min_impurity_decrease=0.3, min_samples_leaf=2,
                       min_weight_fraction_leaf=0.3, n_estimators=70,
                       verbose=2)

In [ ]:
score=cross_val_score(classifier , X_train, y_train ,cv=10)

We find the accuracy by **Cross Val Score Method** :

In [ ]:
score

In [ ]:
score.mean()

OH ! we finally find it. It is not bad but good.

In [ ]:
classifier.fit(X_train, y_train)

We have to find the F1 score so we have to find the predictions.

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
f1_score(y_test, y_pred, average='weighted')

**NOT GOOD BUT STILL OKAY**

We have done with RandomForestClassifier and we are looking forward to DecisionTreeClassifier.
Here is the parameters of Decision Tree Classifier we are going to use random search cv.

In [ ]:
params_decision_tree = {
    'criterion' : ['gini', 'entropy'],
    'splitter'  : ['best', 'random'],
    'max_features' : [0, 1, 2, 3],
    'min_samples_split' : [2, 3, 4 ,5],
    'min_samples_leaf' : [1, 2, 3, 4],
    'random_state' : [0]
}

And the process is same as Random Forest Classifier.

In [ ]:
classifier_decision = DecisionTreeClassifier()

In [ ]:
random_search_dt=RandomizedSearchCV(classifier_decision,
                                 param_distributions=params_decision_tree,
                                 n_iter=10,
                                 scoring='roc_auc',
                                 n_jobs=10,
                                 cv=7,verbose=3)

In [ ]:
start_time = timer(None) 
random_search_dt.fit(X_train,y_train)
timer(start_time) 

In [ ]:
random_search_dt.best_estimator_

In [ ]:
random_search_dt.best_params_

In [ ]:
classifier_decision = DecisionTreeClassifier(criterion='entropy', max_features=2, min_samples_leaf=4,
                       min_samples_split=4, random_state=0, splitter='random')

Now we have done with the decision tree classifier and we find the accuracy scores and then f1 scores.

In [ ]:
score=cross_val_score(classifier_decision , X_train, y_train ,cv=10)

In [ ]:
score

In [ ]:
score.mean()

Oops, 1% loss of accuracy in Decision Tree Classifier. So by looking at accuracy only Random Forest is best than Decision Tree Classifier. But we have to look at F1 scores of Decision Tree.

In [ ]:
classifier_decision.fit(X_train, y_train)

In [ ]:
y_pred_dt = classifier_decision.predict(X_test)

In [ ]:
y_pred_dt

In [ ]:
f1_score(y_test, y_pred_dt, average='weighted')

OH ! F1 score is high. As we all know if a model have highest F1 scores then it will be supposed to be a good model. So as come to the conclusion part, Decision Tree Classifier is suitable for this dataset than the Ranodm Forest Classifier.

Hope you like this notebook very much and upvote if you like this please. Till then **Enjoy Machine Learning**